# 🚀 Alertas de Gols no WhatsApp

In [ ]:

!pip install requests

import requests
import time
from datetime import datetime

# ===========================
# CONFIGURAÇÕES DO USUÁRIO
# ===========================
TOKEN = "COLE_SEU_TOKEN_AQUI"  # Token de Acesso do WhatsApp Cloud API
PHONE_NUMBER_ID = "COLE_SEU_PHONE_NUMBER_ID_AQUI"  # ID do número no WhatsApp API
GROUP_ID = "COLE_SEU_GROUP_ID_AQUI"  # ID do grupo ou número para envio

API_FOOTBALL_KEY = "COLE_SUA_CHAVE_API_FOOTBALL"  # Chave gratuita do API-Football

# Ligas monitoradas (IDs do API-Football)
LEAGUES = {
    "Inglaterra": 39,
    "Noruega": 103,
    "Alemanha": 78,
    "Itália": 135,
    "Portugal": 94,
    "França": 61,
    "Brasil": 71,
    "Argentina": 128,
    "Espanha": 140
}

# ===========================
# FUNÇÃO PARA ENVIAR ALERTA NO WHATSAPP
# ===========================
def send_whatsapp_message(message):
    url = f"https://graph.facebook.com/v18.0/{PHONE_NUMBER_ID}/messages"
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    data = {
        "messaging_product": "whatsapp",
        "to": GROUP_ID,
        "type": "text",
        "text": {"body": message}
    }
    response = requests.post(url, headers=headers, json=data)
    print("Mensagem enviada:", response.json())

# ===========================
# FUNÇÃO PARA VERIFICAR JOGOS AO VIVO
# ===========================
def check_live_matches():
    url = "https://v3.football.api-sports.io/fixtures"
    headers = {"x-apisports-key": API_FOOTBALL_KEY}
    today = datetime.now().strftime("%Y-%m-%d")

    for league_name, league_id in LEAGUES.items():
        params = {"date": today, "live": "all", "league": league_id}
        r = requests.get(url, headers=headers, params=params).json()
        
        if "response" in r and r["response"]:
            for match in r["response"]:
                home = match["teams"]["home"]["name"]
                away = match["teams"]["away"]["name"]
                score_home = match["goals"]["home"]
                score_away = match["goals"]["away"]

                if score_home + score_away < 3:
                    msg = f"⚽ Possível aposta em GOL!\n{home} x {away}\nPlacar: {score_home} - {score_away}"
                    send_whatsapp_message(msg)

print("📡 Iniciando monitoramento de jogos ao vivo...")
while True:
    check_live_matches()
    time.sleep(60)
